In this notebook we try to predict fumigation status using the microbiome data.

In [2]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [19]:
# read in data
df_abundance = pd.read_csv('bac_abundance.csv')
metadata = pd.read_csv("samp_metadata.csv")

In [20]:
df_abundance.drop(columns=['Unnamed: 0'], inplace=True)
df_family = df_abundance.groupby('Family').sum()
df_family.head()

C:\Users\tiany\AppData\Local\Temp\ipykernel_26008\2874980339.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_family = df_abundance.groupby('Family').sum()


,Samp001,Samp002,Samp003,Samp004,Samp005,Samp006,Samp007,Samp008,Samp009,Samp010,...,Samp231,Samp232,Samp233,Samp234,Samp235,Samp236,Samp237,Samp238,Samp239,Samp240
Family,,,,,,,,,,,,,,,,,,,,,
01D2Z36,1,0,0,0,0,0,0,0,2,0,...,17,1,6,3,4,7,4,5,1,5
0319-6G20,4,0,4,5,4,0,10,3,3,8,...,15,10,19,8,20,30,18,16,28,27
053A03-B-DI-P58,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
27F-1492R,0,0,1,0,0,0,1,1,0,0,...,0,0,0,3,1,6,1,1,3,1
37-13,4,0,3,2,0,2,6,3,0,7,...,15,8,20,8,9,20,6,28,11,15


In [21]:
metadata.drop(columns=['Unnamed: 0'], inplace=True)
metadata.head()

,Samp,samp_number,Time,Treatment,Date
0,Fum-1-1,1,Day_0,Non-amended,2016-08-08
1,Fum-1-2,2,Day_0,Non-amended,2016-08-08
2,Fum-1-3,3,Day_0,Non-amended,2016-08-08
3,Fum-1-4,4,Day_0,Non-amended,2016-08-08
4,Fum-1-5,5,Day_0,Non-amended,2016-08-08


In [22]:
metadata

,Samp,samp_number,Time,Treatment,Date
0,Fum-1-1,1,Day_0,Non-amended,2016-08-08
1,Fum-1-2,2,Day_0,Non-amended,2016-08-08
2,Fum-1-3,3,Day_0,Non-amended,2016-08-08
3,Fum-1-4,4,Day_0,Non-amended,2016-08-08
4,Fum-1-5,5,Day_0,Non-amended,2016-08-08
...,...,...,...,...,...
235,Fum-1-236,236,Day_282,Non-fumigated chipping grass,2017-05-17
236,Fum-1-237,237,Day_282,Non-fumigated chipping grass,2017-05-17
237,Fum-1-238,238,Day_282,Non-fumigated chipping grass,2017-05-17
238,Fum-1-239,239,Day_282,Non-fumigated chipping grass,2017-05-17


In [26]:
# 0 = non-fumigated, 1 = recently fumigated, 2 = fumigated more than a month ago
metadata["Fumigation"] = None

for i, row in enumerate(metadata.itertuples()):
    if row.Time == "Day_0" or row.Treatment == "Non-fumigated chipping grass":
        metadata.at[i, "Fumigation"] = 0
    elif row.Time == "Day_10" and row.Treatment != "Non-fumigated chipping grass":
        metadata.at[i, "Fumigation"] = 1
    else:
        metadata.at[i, "Fumigation"] = 2

metadata


,Samp,samp_number,Time,Treatment,Date,Fumigation
0,Fum-1-1,1,Day_0,Non-amended,2016-08-08,0
1,Fum-1-2,2,Day_0,Non-amended,2016-08-08,0
2,Fum-1-3,3,Day_0,Non-amended,2016-08-08,0
3,Fum-1-4,4,Day_0,Non-amended,2016-08-08,0
4,Fum-1-5,5,Day_0,Non-amended,2016-08-08,0
...,...,...,...,...,...,...
235,Fum-1-236,236,Day_282,Non-fumigated chipping grass,2017-05-17,0
236,Fum-1-237,237,Day_282,Non-fumigated chipping grass,2017-05-17,0
237,Fum-1-238,238,Day_282,Non-fumigated chipping grass,2017-05-17,0
238,Fum-1-239,239,Day_282,Non-fumigated chipping grass,2017-05-17,0


In [40]:
# Get X and y
X = df_family.T.values
y = metadata["Fumigation"].values.astype(int)

In [41]:
# import more libraries
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold
from xgboost import XGBClassifier

# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# create model
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# cv
kfold = RepeatedStratifiedKFold(n_splits=5, n_repeats=40, random_state=42)
scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=kfold, n_jobs=-1)

# Print the mean and standard deviation of the scores
print("Cross-validated Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


c:\Users\tiany\anaconda3\Lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
c:\Users\tiany\anaconda3\Lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
c:\Users\tiany\anaconda3\Lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
c:\Users\tiany\anaconda3\Lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
c:\Users\tiany\anaconda3\Lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
c:\Users\tiany\anaconda3\Lib\site-packages\xgboost\sklearn.p

Cross-validated Accuracy: 0.99 (+/- 0.04)


In [42]:
# test model
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
test_score = model.score(X_test, y_test)
print("Test Accuracy: %0.2f" % test_score)

Test Accuracy: 0.96
